In [1]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np

# Load the pre-trained model (best_model.pkl)
with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)

# Function to make prediction
def predict_alzheimers(features):
    prediction = model.predict([features])
    probability = model.predict_proba([features])[0][1]  # Probability of the positive class (Alzheimer's)
    return prediction, probability

# Function to clean the data and handle non-numeric values
def clean_data(df):
    # Convert columns that should be numeric to float, replace invalid values with NaN
    df = df.apply(pd.to_numeric, errors='coerce')

    # Fill NaN values with the column mean or median (you can customize this)
    df = df.fillna(df.mean())

    return df

# Streamlit app UI
st.title("Alzheimer's Disease Prediction")
st.write("Upload a CSV file and select a patient to predict Alzheimer's Disease.")

# Upload CSV file
uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

if uploaded_file is not None:
    # Read the CSV file
    df = pd.read_csv(uploaded_file)

    # Display the first few rows of the dataset for the user to inspect
    st.write("Dataset preview:")
    st.write(df.head())

    # Ensure there's a unique identifier column like 'PatientID' or 'Name'
    if 'PatientID' in df.columns:
        patient_id_column = 'PatientID'
    elif 'Name' in df.columns:
        patient_id_column = 'Name'
    else:
        st.error("No PatientID or Name column found in the dataset!")
        patient_id_column = None

    # If there's a valid patient identifier column, let the user select a patient
    if patient_id_column:
        patient_id = st.selectbox("Select a Patient", df[patient_id_column].unique())

        # Get the row corresponding to the selected patient
        selected_patient = df[df[patient_id_column] == patient_id]

        # Extract the features for prediction (excluding the target column 'Diagnosis')
        features = selected_patient.drop(columns=['Diagnosis'])

        # Clean the data by converting all columns to numeric and handling errors
        features = clean_data(features)

        # Convert the cleaned features to a list of feature values
        feature_values = features.iloc[0].values.tolist()

        # Predict button
        if st.button('Predict'):
            prediction, probability = predict_alzheimers(feature_values)
            if prediction == 1:
                st.write(f"The patient {patient_id} is predicted to have Alzheimer's disease with a {probability*100:.2f}% chance.")
            else:
                st.write(f"The patient {patient_id} is predicted not to have Alzheimer's disease with a {100 - probability*100:.2f}% chance.")


2025-03-10 14:08:02.886 
  command:

    streamlit run C:\Users\cheta\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
